#  dataset Eurostat e dimensioni 
(geo, time, unit, ecc.):


Eurostat offre un file `.txt` contenente l’inventario completo dei dataset con informazioni sul titolo, codice e gerarchia:

* URL: `https://ec.europa.eu/eurostat/api/dissemination/catalogue/toc/txt?lang=en`
* È un file delimitato da tabulazioni, facilmente leggibile con Pandas.



[1]: https://ec.europa.eu/eurostat/data/web-services?utm_source=chatgpt.com "Web services - Eurostat - European Commission"
[2]: https://ec.europa.eu/eurostat/web/user-guides/data-browser/api-data-access/api-getting-started/catalogue-api?utm_source=chatgpt.com "Getting started with catalogue API - User guides - Eurostat"
[3]: https://ec.europa.eu/eurostat/web/user-guides/data-browser/api-data-access/api-getting-started/api?utm_source=chatgpt.com "Getting started with statistics API - User guides - Eurostat"
[4]: https://ec.europa.eu/eurostat/web/user-guides/data-browser/api-data-access/api-detailed-guidelines/api-statistics?utm_source=chatgpt.com "Detailed guidelines - API Statistics - User guides - Eurostat"


In [1]:
import pandas as pd

# 1️⃣ Scarica e carica il TOC
toc = pd.read_csv(
    "https://ec.europa.eu/eurostat/api/dissemination/catalogue/toc/txt?lang=en",
    sep='\t'
)

# 2️⃣ Filtra solo i dataset (esclude folder/table)
datasets = toc[toc['type'] == 'dataset'].copy()

# 3️⃣ Per ogni dataset, estrai le dimensioni via API Statistiche (JSON-stat)
import requests

def get_dimensions(ds):
    url = f"https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/{ds}?format=JSON"
    resp = requests.get(url)
    if resp.status_code != 200:
        return None
    meta = resp.json().get('id', [])
    return '|'.join(meta)

datasets['dimensions'] = datasets['code'].apply(get_dimensions)

# 4️⃣ Salva in CSV
datasets[['code', 'title', 'dimensions']].to_csv("D:/eurostat_datasets_dims.csv", index=False)
print("✅ File generato: eurostat_datasets_dims.csv")
